In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import json
from fuzzywuzzy import fuzz
import uuid

In [2]:
def pprint(dict, indent=4):
    print(json.dumps(dict, indent=indent))

In [3]:
"""
Load the data extracted for level 1
Level 1 data consists of the models for each of the manufacturer
This will be used to perform the matching with the existing dataset (Model Gender dataset) that is present in the 
excel sheet provided and further scraping will only be done on the models that are present in the dataset
"""

with open('cars_data_level1.json') as f:
    cars_data_level1 = json.load(f)
    manufacturers = list(cars_data_level1.keys())

print(manufacturers)
pprint(cars_data_level1, indent=2)

['Abarth', 'Aiways', 'Alfa Romeo', 'Alpine', 'Asia Motors', 'Aston Martin', 'Audi', 'Austin', 'Autobianchi', 'Bentley', 'BMW', 'Bugatti', 'Buick', 'Cadillac', 'Carver', 'Chevrolet', 'Chrysler', 'Citroen', 'Corvette', 'Cupra', 'Dacia', 'Daewoo', 'Daihatsu', 'Daimler', 'Datsun', 'Dodge', 'Donkervoort', 'DS', 'Ferrari', 'Fiat', 'Fisker', 'Ford', 'FSO', 'Galloper', 'Honda', 'Hummer', 'Hyundai', 'Infiniti', 'Innocenti', 'Iveco', 'Jaguar', 'Jeep', 'Josse', 'Kia', 'Lada', 'Lamborghini', 'Lancia', 'Land Rover', 'Landwind', 'Lexus', 'Lincoln', 'Lotus', 'Marcos', 'Maserati', 'Maybach', 'Mazda', 'McLaren', 'Mega', 'Mercedes-Benz', 'Mercury', 'MG', 'Mini', 'Mitsubishi', 'Morgan', 'Morris', 'Nissan', 'Noble', 'Opel', 'Peugeot', 'PGO', 'Polestar', 'Pontiac', 'Porsche', 'Princess', 'Renault', 'Rolls-Royce', 'Rover', 'Saab', 'Seat', 'Skoda', 'Smart', 'Spectre', 'Ssangyong', 'Subaru', 'Suzuki', 'Talbot', 'Tesla', 'Think', 'Toyota', 'Triumph', 'TVR', 'Volkswagen', 'Volvo', 'Yugo']
{
  "Abarth": {
    "A

In [4]:
"""
Preprocessing of all the manufacturers and cars_data_level1

1. Remove '-' from the manufacturer name otherwise it will be an issue when matching name with the models available in the dataset
2. Convert all the names to lowercase
"""

def preprocess_manufacturers(manufacturers):
    # Preprocess all the manufacturers names to remove '-' from their names and convert 
    return list(map(lambda manufacturer: manufacturer.replace("-"," ").upper(), manufacturers))

def preprocess_cars_data_level1(cars_data_level1):
    all_cars_data = {}
    for manufacturer, models in cars_data_level1.items():
        manufacturer = manufacturer.upper()
        all_cars_data[manufacturer] = {}
        for model in models:
            all_cars_data[manufacturer][model.upper()] = models[model]
    return all_cars_data

manufacturers = preprocess_manufacturers(manufacturers)
cars_data_level1 = preprocess_cars_data_level1(cars_data_level1)
print(manufacturers)
pprint(cars_data_level1)

['ABARTH', 'AIWAYS', 'ALFA ROMEO', 'ALPINE', 'ASIA MOTORS', 'ASTON MARTIN', 'AUDI', 'AUSTIN', 'AUTOBIANCHI', 'BENTLEY', 'BMW', 'BUGATTI', 'BUICK', 'CADILLAC', 'CARVER', 'CHEVROLET', 'CHRYSLER', 'CITROEN', 'CORVETTE', 'CUPRA', 'DACIA', 'DAEWOO', 'DAIHATSU', 'DAIMLER', 'DATSUN', 'DODGE', 'DONKERVOORT', 'DS', 'FERRARI', 'FIAT', 'FISKER', 'FORD', 'FSO', 'GALLOPER', 'HONDA', 'HUMMER', 'HYUNDAI', 'INFINITI', 'INNOCENTI', 'IVECO', 'JAGUAR', 'JEEP', 'JOSSE', 'KIA', 'LADA', 'LAMBORGHINI', 'LANCIA', 'LAND ROVER', 'LANDWIND', 'LEXUS', 'LINCOLN', 'LOTUS', 'MARCOS', 'MASERATI', 'MAYBACH', 'MAZDA', 'MCLAREN', 'MEGA', 'MERCEDES BENZ', 'MERCURY', 'MG', 'MINI', 'MITSUBISHI', 'MORGAN', 'MORRIS', 'NISSAN', 'NOBLE', 'OPEL', 'PEUGEOT', 'PGO', 'POLESTAR', 'PONTIAC', 'PORSCHE', 'PRINCESS', 'RENAULT', 'ROLLS ROYCE', 'ROVER', 'SAAB', 'SEAT', 'SKODA', 'SMART', 'SPECTRE', 'SSANGYONG', 'SUBARU', 'SUZUKI', 'TALBOT', 'TESLA', 'THINK', 'TOYOTA', 'TRIUMPH', 'TVR', 'VOLKSWAGEN', 'VOLVO', 'YUGO']
{
    "ABARTH": {
    

In [5]:
"""
Load the dataset using pandas and display top 10 rows
"""
dataset = pd.read_csv('./150106-D-private Cars lic 1995 to 2013 Model gender-MD.csv')
dataset.head(10)

,Year,Make/Model,Male,Female,Unknown,Total,Male.1,Female.1,Unknown.1
0,2013,ABARTH 500,2305,1697,180,4182,55%,41%,4%
1,2012,ABARTH 500,2081,1373,155,3609,58%,38%,4%
2,2011,ABARTH 500,1637,1000,142,2779,59%,36%,5%
3,2010,ABARTH 500,1101,649,93,1843,60%,35%,5%
4,2009,ABARTH 500,606,280,38,924,66%,30%,4%
5,2013,ABARTH 595,411,224,26,661,62%,34%,4%
6,2012,ABARTH 595,89,31,0,120,74%,26%,0%
7,2013,ABARTH 695,1,0,0,1,100%,0%,0%
8,2013,ABARTH Model Missing,14,4,0,18,78%,22%,0%
9,2012,ABARTH Model Missing,11,4,0,15,73%,27%,0%


In [6]:
"""
Consider only the models with total sales >= 1000 for our further analysis.
A total of 847 models are left after this preprocessing step
"""
SALES_THRESHOLD = 1000
dataset_models = dataset.loc[dataset['Total'] >= SALES_THRESHOLD, 'Make/Model'].unique()
print(len(dataset_models))
print(dataset_models)

847
['ABARTH 500' 'ALFA ROMEO 145' 'ALFA ROMEO 146' 'ALFA ROMEO 147'
 'ALFA ROMEO 155' 'ALFA ROMEO 156' 'ALFA ROMEO 159' 'ALFA ROMEO 164'
 'ALFA ROMEO 166' 'ALFA ROMEO 33' 'ALFA ROMEO 75' 'ALFA ROMEO ALFASUD'
 'ALFA ROMEO BRERA' 'ALFA ROMEO GIULIETTA' 'ALFA ROMEO GT'
 'ALFA ROMEO GTV' 'ALFA ROMEO MITO' 'ALFA ROMEO Model Missing'
 'ALFA ROMEO SPIDER' 'ALFA ROMEO SPRINT' 'ASTON MARTIN DB7'
 'ASTON MARTIN DB9' 'ASTON MARTIN VANTAGE' 'AUDI 100' 'AUDI 200' 'AUDI 80'
 'AUDI 90' 'AUDI A1' 'AUDI A2' 'AUDI A3' 'AUDI A4' 'AUDI A5' 'AUDI A6'
 'AUDI A7' 'AUDI A8' 'AUDI ALLROAD' 'AUDI C' 'AUDI CABRIOLET' 'AUDI COUPE'
 'AUDI Model Missing' 'AUDI Q3' 'AUDI Q5' 'AUDI Q7' 'AUDI QUATTRO'
 'AUDI R8' 'AUDI RIO' 'AUDI RS4' 'AUDI RS6' 'AUDI S3' 'AUDI S4' 'AUDI S5'
 'AUDI TT' 'AUSTIN 1100' 'AUSTIN 1300' 'AUSTIN A35' 'AUSTIN A40'
 'AUSTIN ALLEGRO' 'AUSTIN AMBASSADOR' 'AUSTIN HEALEY'
 'AUSTIN HEALEY Model Missing' 'AUSTIN MAESTRO' 'AUSTIN MAXI'
 'AUSTIN METRO' 'AUSTIN MINI' 'AUSTIN Model Missing' 'AUSTIN MONTE

In [7]:
"""
As can be seen in the above list, some of the model names are missing (example: AUDI Model Missing)
We skip these missing models and are left with 794 models.
"""
dataset_models = np.array(list(filter(lambda x: not 'model missing' in x.lower(), dataset_models)))
print(len(dataset_models))
print(dataset_models)

794
['ABARTH 500' 'ALFA ROMEO 145' 'ALFA ROMEO 146' 'ALFA ROMEO 147'
 'ALFA ROMEO 155' 'ALFA ROMEO 156' 'ALFA ROMEO 159' 'ALFA ROMEO 164'
 'ALFA ROMEO 166' 'ALFA ROMEO 33' 'ALFA ROMEO 75' 'ALFA ROMEO ALFASUD'
 'ALFA ROMEO BRERA' 'ALFA ROMEO GIULIETTA' 'ALFA ROMEO GT'
 'ALFA ROMEO GTV' 'ALFA ROMEO MITO' 'ALFA ROMEO SPIDER'
 'ALFA ROMEO SPRINT' 'ASTON MARTIN DB7' 'ASTON MARTIN DB9'
 'ASTON MARTIN VANTAGE' 'AUDI 100' 'AUDI 200' 'AUDI 80' 'AUDI 90'
 'AUDI A1' 'AUDI A2' 'AUDI A3' 'AUDI A4' 'AUDI A5' 'AUDI A6' 'AUDI A7'
 'AUDI A8' 'AUDI ALLROAD' 'AUDI C' 'AUDI CABRIOLET' 'AUDI COUPE' 'AUDI Q3'
 'AUDI Q5' 'AUDI Q7' 'AUDI QUATTRO' 'AUDI R8' 'AUDI RIO' 'AUDI RS4'
 'AUDI RS6' 'AUDI S3' 'AUDI S4' 'AUDI S5' 'AUDI TT' 'AUSTIN 1100'
 'AUSTIN 1300' 'AUSTIN A35' 'AUSTIN A40' 'AUSTIN ALLEGRO'
 'AUSTIN AMBASSADOR' 'AUSTIN HEALEY' 'AUSTIN MAESTRO' 'AUSTIN MAXI'
 'AUSTIN METRO' 'AUSTIN MINI' 'AUSTIN MONTEGO' 'AUSTIN MORRIS MAXI'
 'AUSTIN MORRIS MINI' 'AUSTIN MORRIS PRINCESS' 'AUSTIN SEVEN'
 'BENTLEY ARNAG

In [8]:
"""
LEVEL 1 MATCHING:

Obtain manufacturers of all the models present in the dataset
using the manufacturers list obtained from the scraped data

One obvious way to do this is to check if the name of the manufacturer is present in the name of the model
However based on the data obtained, this is not the right approach

The possible areas of conflict can arise because of the following kind of examples:

1. VAUXHALL OMEGA (Contains name of two manufacturers MEGA and VAUXHALL)
2. AUSTIN MINI (Contains name of two manufacturers AUSTON and MINI)
3. AUSTIN MORRIS MAXI (Contains name of two manufacturers AUSTON and MAXI)
4. CHEVROLET GMC CORVETTE (Contains name of two manufacturers CHEVROLET and CORVETTE)

etc.

Hence in order to resolve this, we check if the name of the model "starts" with the manufacturer
Which resolves all the corner cases we've encountered above
"""
pd.set_option('display.max_rows', 1000)

company_map = {}
for model in dataset_models:
    for manufacturer in manufacturers:
        try:
            if model.index(manufacturer) == 0:
                company_map[model] = manufacturer
        except:
            pass

# Print model names with their manufacturers name as obtained after the preprocessing step
pd.DataFrame({'Model': list(company_map.keys()), 'Manufacturer': list(company_map.values())})

,Model,Manufacturer
0,ABARTH 500,ABARTH
1,ALFA ROMEO 145,ALFA ROMEO
2,ALFA ROMEO 146,ALFA ROMEO
3,ALFA ROMEO 147,ALFA ROMEO
4,ALFA ROMEO 155,ALFA ROMEO
5,ALFA ROMEO 156,ALFA ROMEO
6,ALFA ROMEO 159,ALFA ROMEO
7,ALFA ROMEO 164,ALFA ROMEO
8,ALFA ROMEO 166,ALFA ROMEO
9,ALFA ROMEO 33,ALFA ROMEO


In [9]:
"""
LEVEL 2 MATCHING:
For all the models obtained till now, match it with the model available on the website
This is required for scraping purposes in order to extract only the "relevant data"
Matching the model names with the ones from the websites is tricky as there are instances
where we may not be able to obtain an exact match. We resolve this by using "fuzzy search" methodology
1st priority is given to fuzzy token_sort_ratio value
in case of equal token_sort_ratio, 2nd priority is given to fuzzy ratio
"""

web_scarping_prior_data = []
for model, manufacturer in company_map.items():
    ratios = []
    models = []
    for key in cars_data_level1[manufacturer]:
        ratio1 = fuzz.ratio(model, key)
        ratio2 = fuzz.token_sort_ratio(model, key)
        ratios.append((ratio2, ratio1))
        models.append(key)
    max_ratio = (0, 0)
    matched_model = ""
    for i, ratio in enumerate(ratios):
        max_ratio = max(ratio, max_ratio)
        if (ratio == max_ratio):
            matched_model = models[i]
    if matched_model != "" and max_ratio[0] > 90:
        if model != matched_model:
            print(f'Model: {model} | Matched model: {matched_model} | Max ratio: {max_ratio}')
        web_scarping_prior_data.append((model, matched_model, manufacturer, cars_data_level1[manufacturer][matched_model]['link']))

Model: BMW 1 SERIES | Matched model: BMW 1-SERIES | Max ratio: (100, 92)
Model: BMW 3 SERIES | Matched model: BMW 3-SERIES | Max ratio: (100, 92)
Model: BMW 5 SERIES | Matched model: BMW 5-SERIES | Max ratio: (100, 92)
Model: BMW 6 SERIES | Matched model: BMW 6-SERIES | Max ratio: (100, 92)
Model: BMW 7 SERIES | Matched model: BMW 7-SERIES | Max ratio: (100, 92)
Model: BMW 8 SERIES | Matched model: BMW 8-SERIES | Max ratio: (100, 92)
Model: CHRYSLER 300 | Matched model: CHRYSLER 300M | Max ratio: (96, 96)
Model: DAIHATSU GRAND MOVE | Matched model: DAIHATSU GRAN MOVE | Max ratio: (97, 97)
Model: DAIMLER 4.2 | Matched model: DAIMLER 4.0 | Max ratio: (91, 91)
Model: HYUNDAI ATOZ | Matched model: HYUNDAI ATOS | Max ratio: (92, 92)
Model: HYUNDAI S-COUPE | Matched model: HYUNDAI COUPE | Max ratio: (93, 93)
Model: HYUNDAI X2 | Matched model: HYUNDAI IX20 | Max ratio: (91, 91)
Model: JAGUAR S TYPE | Matched model: JAGUAR S-TYPE | Max ratio: (100, 92)
Model: JAGUAR X TYPE | Matched model: JAG

In [10]:
"""
Removing the extreme outliers:

As seen in the above output, the following matches are outliers and can be removed

1. Model: HYUNDAI X2 | Matched model: HYUNDAI IX20 | Max ratio: (91, 91)
2. Model: VOLKSWAGEN URBAN | Matched model: VOLKSWAGEN TOURAN | Max ratio: (91, 91)

We can also see that using fuzzy search method has helped find out the model names such as
1. Model: HYUNDAI ATOZ | Matched model: HYUNDAI ATOS | Max ratio: (92, 92)
2. Model: TVR CHIMERA | Matched model: TVR CHIMAERA | Max ratio: (96, 96)
etc.

We will relax the restrictions for the below values
1. Model: HYUNDAI S-COUPE | Matched model: HYUNDAI COUPE | Max ratio: (93, 93)
2. Model: JAGUAR XK8 | Matched model: JAGUAR XK | Max ratio: (95, 95)
3. Model: JAGUAR XKR | Matched model: JAGUAR XK | Max ratio: (95, 95)

which otherwise would be missed if we use direct string matching
"""

scraping_data = pd.DataFrame(web_scarping_prior_data, columns=['Model', 'Matched Model', 'Manufacturer', 'Link'])
display(scraping_data)

,Model,Matched Model,Manufacturer,Link
0,ABARTH 500,ABARTH 500,ABARTH,https://www.cars-data.com/en/abarth/500
1,ALFA ROMEO 145,ALFA ROMEO 145,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/145
2,ALFA ROMEO 146,ALFA ROMEO 146,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/146
3,ALFA ROMEO 147,ALFA ROMEO 147,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/147
4,ALFA ROMEO 155,ALFA ROMEO 155,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/155
5,ALFA ROMEO 156,ALFA ROMEO 156,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/156
6,ALFA ROMEO 159,ALFA ROMEO 159,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/159
7,ALFA ROMEO 164,ALFA ROMEO 164,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/164
8,ALFA ROMEO 166,ALFA ROMEO 166,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/166
9,ALFA ROMEO 33,ALFA ROMEO 33,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/33


In [11]:
"""
START WEB SCRAPING
"""

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import time
from webdriver_manager.chrome import ChromeDriverManager
import pickle5 as pickle
import os
import signal
import json

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(10)

In [12]:
"""
Handling web scraping challenges:

There are times that the script will give 403 forbidden or will timeout due to excessive scraping
In such a scenario, we would not want to lose out the data that we've already extracted
Hence, manually feed the value of `last_scraped` to be the model name which was successfully scraped (latest)
The script will resume after this model. If starting from the beginning, put the value to be an empty string
"""

last_scraped = ''

In [13]:
start_idx = 0
if last_scraped != '':
    with open('scraped_cars_data.pickle','rb') as f:
        scraped_cars_data = pickle.load(f)
    start_idx = scraping_data.Models.searchsorted(last_scraped)

In [14]:
scraping_data = scraping_data[start_idx:]

In [15]:
def get_year_data_for_current_year():
    title = driver.find_element_by_class_name('title')
    description = title.find_element_by_xpath('div[2]/h2').get_attribute('innerText')
    divs = driver.find_element_by_xpath(
        '/html/body/main/div/div[2]/section[2]/div[1]')
    types = divs.find_elements_by_class_name('row')
    types_data = []
    for type in types:
        columns = type.find_elements_by_tag_name('div')
        data = {}
        data['link'] = columns[0].find_element_by_tag_name(
            'a').get_attribute('href')
        data['title'] = columns[0].text
        data['fuel'] = columns[1].text
        data['transmission'] = columns[2].text
        data['energy'] = columns[3].text
        types_data.append(data)
    year_data = {}
    year_data['description'] = description
    year_data['types'] = types_data
    return year_data

def get_years_data_from_year_links(year_links):
    years = sorted(list(year_links.keys()))
    print(years)
    years_data = {}
    for year in years:
        # AN OPTIMIZATION TO SAVE SCRAPING TIME
        # The link corresponding to the least year is already opened
        # do not open it again
        link = year_links[year]
        if link != driver.current_url:
            driver.get(link)
        years_data[year] = get_year_data_for_current_year()
    return years_data

def get_model_types(model_data):
    link = model_data['link']
    driver.get(link)
    
    # Extract all the links corresponding to all the years
    element = driver.find_element_by_xpath(
        '/html/body/main/div/div[2]/section[2]/div[1]')
    divs = element.find_elements_by_tag_name('div')
    years = divs[0].find_elements_by_tag_name('a')
    year_links = {}
    for year in years:
        year_links[year.get_attribute('innerText')] = year.get_attribute('href')
    
    # Obtain all the images from the current page
    image_guids = []
    images_divs = driver.find_elements_by_class_name('img100')
    for images_div in images_divs:
        images = images_div.find_elements_by_xpath('div/a/picture/img')
        for image in images:
            guid = uuid.uuid4().hex
            src = image.get_attribute('src')
            scraped_static_data[guid] = src
            image_guids.append(guid)
    
    return get_years_data_from_year_links(year_links), image_guids

def get_models_data(models, recurse=True):
    models_data = []
    # Extract information corresponding to all the models
    for model in models:
        model_data = {}
        model_data['title'] = model.get_attribute('innerText')
        model_data['link']  = model.get_attribute('href')
        model_data['years'] = {}
        models_data.append(model_data)
    if not recurse:
        return models_data
    
    # Recursively scrape information for all the models
    # launched in all the years
    for idx, model_data in enumerate(models_data):
        model_data['years'], model_data['images'] = get_model_types(model_data)
    return models_data

scraped_cars_data = {}
scraped_static_data = {}

for data in scraping_data.iloc:
    display(pd.DataFrame(data).T)
    
    # Obtain the link for the current model
    link = data['Link']
    
    # Open the link in the browser
    driver.get(link)
    
    # Find all the models for the current model
    models_elements = driver.find_elements_by_class_name('models')
    models = []
    for model in models_elements[:-1]:
        models += model.find_elements_by_xpath('./div/a')
    # Obtain the data (recursively can be True/False) for all the models
    models_data = get_models_data(models)
    scraped_cars_data[data['Model']] = models_data

,Model,Matched Model,Manufacturer,Link
0,ABARTH 500,ABARTH 500,ABARTH,https://www.cars-data.com/en/abarth/500


['2008', '2012', '2014', '2015']


,Model,Matched Model,Manufacturer,Link
1,ALFA ROMEO 145,ALFA ROMEO 145,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/145


['1999']
['1994', '1995', '1996', '1997', '1998']


,Model,Matched Model,Manufacturer,Link
2,ALFA ROMEO 146,ALFA ROMEO 146,ALFA ROMEO,https://www.cars-data.com/en/alfa-romeo/146


...

['2017', '2018', '2019', '2020']
['2013', '2014', '2015', '2016']
['2008', '2009', '2010', '2011', '2012', '2013']


,Model,Matched Model,Manufacturer,Link
533,VOLVO XC70,VOLVO XC70,VOLVO,https://www.cars-data.com/en/volvo/xc70


['2013', '2014', '2015', '2016']
['2011', '2012', '2013']
['2007', '2008', '2009', '2010']
['2004', '2005']
['2002', '2004']


,Model,Matched Model,Manufacturer,Link
534,VOLVO XC90,VOLVO XC90,VOLVO,https://www.cars-data.com/en/volvo/xc90


['2019', '2020']
['2014', '2016', '2017']
['2011', '2012']
['2006', '2008', '2010', '2011']
['2002', '2003', '2004', '2005']


In [16]:
pprint(scraped_cars_data, indent=2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
pprint(scraped_static_data, indent=2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
with open('scraped_cars_data.pickle', 'wb') as f:
    pickle.dump(scraped_cars_data, f)

In [19]:
with open('scraped_static_data.pickle', 'wb') as f:
    pickle.dump(scraped_static_data, f)